In [3]:
import numpy as np
import pandas as pd
from collections import Counter

from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, accuracy_score

import warnings
warnings.filterwarnings('ignore')

# --- 1. 加载数据 ---
# 加载训练集、测试集和外部原始数据集。
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
orig_df = pd.read_csv('personality_datasert.csv')

# --- 2. 特征工程 ---
# 定义用于合并的列。
df_cols = [
    'Time_spent_Alone', 'Stage_fear', 'Social_event_attendance',
    'Going_outside', 'Drained_after_socializing',
    'Friends_circle_size', 'Post_frequency'
]

# 通过重命名列和删除重复项来准备原始数据集。
# 这会创建一个参考数据帧，用于识别训练/测试集中的匹配记录。
ref_df = (
    orig_df.rename(columns={'Personality': 'match_p'})
           .drop_duplicates(subset=df_cols)
)

# 用于将主数据帧与参考数据帧合并的函数。
# 它会添加一个 'match_p' 列（原始的人格标签）和一个标志 'match_p_null'
# 来指示是否在原始数据集中找到了匹配项。
def merge_with_match_p(df, ref_df, merge_cols):
    merged_df = df.merge(ref_df, how='left', on=merge_cols)
    merged_df['match_p_null'] = merged_df['match_p'].isna().astype(int)
    return merged_df

# 将合并函数应用于训练集和测试集。
train = merge_with_match_p(train_df, ref_df, df_cols)
test = merge_with_match_p(test_df, ref_df, df_cols)

# 删除合并过程中可能产生的任何重复行。
train = train.drop_duplicates(subset=['id'], keep='first')
test = test.drop_duplicates(subset=['id'], keep='first')

# 用 'unknown' 填充新的 'match_p' 列中的缺失值。
train['match_p'] = train['match_p'].fillna('unknown')
test['match_p'] = test['match_p'].fillna('unknown')

# --- 3. 数据预处理 ---
# 将目标变量 'Personality' 转换为二进制（1 代表外向，0 代表内向）。
train['Personality'] = train['Personality'].map({'Extrovert': 1, 'Introvert': 0})

# 定义要编码的分类列。
cat_cols = ['match_p', 'Stage_fear', 'Drained_after_socializing']

# 用于将分类字符串值映射为数值的函数。
def cat_encode(df, columns):
    for col in columns:
        if col == 'match_p':
            df[col] = df[col].map({'Extrovert': 2, 'Introvert': 1, 'unknown': 0})
        else:
            df[col] = df[col].map({'Yes': 1, 'No': 0})
    return df

# 将编码应用于训练集和测试集。
train = cat_encode(train, cat_cols)
test = cat_encode(test, cat_cols)

# 使用每列的平均值来填补数值列中的缺失值。
for col in train.drop(columns=['id', 'Personality']).columns:
    train[col] = train[col].fillna(train[col].mean())

for col in test.drop(columns=['id']).columns:
    test[col] = test[col].fillna(test[col].mean())

# --- 4. 准备模型数据 ---
# 分离特征（X）和目标（y）。
X = train.drop(['Personality', 'id'], axis=1)
y = train['Personality']
X_test = test.drop('id', axis=1)

# 计算 'scale_pos_weight' 以处理某些模型中的类别不平衡问题。
# 这会在训练期间为少数类别提供更多权重。
counter = Counter(y)
scale_pos_weight = counter[0] / counter[1]

# --- 5. 模型配置 ---
# 为基础模型定义超参数。
catboost_params = {
    'iterations': 1000, 'learning_rate': 0.001, 'depth': 8, 'l2_leaf_reg': 3,
    'border_count': 128, 'bagging_temperature': 1, 'random_strength': 1,
    'loss_function': 'Logloss', 'eval_metric': 'AUC', 'scale_pos_weight': scale_pos_weight,
    'verbose': 0, 'random_state': 42
}

lgb_params = {
    'objective': 'binary', 'n_estimators': 300, 'max_depth': 10, 'min_child_samples': 10,
    'num_leaves': 20, 'learning_rate': 0.0109, 'colsample_bytree': 0.8819,
    'subsample': 0.7015, 'scale_pos_weight': scale_pos_weight, 'metric': 'AUC',
    'random_state': 42, 'verbosity': 0
}

xgb_params = {
    'n_estimators': 642, 'learning_rate': 0.1337, 'max_depth': 8, 'subsample': 0.5003,
    'colsample_bytree': 0.5300, 'gamma': 2.3324, 'reg_lambda': 2.4718, 'reg_alpha': 0.1671,
    'min_child_weight': 8, 'objective': 'binary:logistic', 'eval_metric': 'auc',
    'use_label_encoder': False, 'scale_pos_weight': scale_pos_weight, 'random_state': 42
}

rf_params = {
    'n_estimators': 608, 'max_depth': 9, 'min_samples_split': 17, 'min_samples_leaf': 2,
    'max_features': 'log2', 'bootstrap': True, 'class_weight': 'balanced', 'random_state': 42
}

# --- 6. Stacking 集成模型 ---
# 定义基础模型（第 0 层）。
estimators = [
    ('lgb', LGBMClassifier(**lgb_params)),
    ('rf', RandomForestClassifier(**rf_params)),
    ('xgb', XGBClassifier(**xgb_params)),
    ('cat', CatBoostClassifier(**catboost_params))
]

# 定义元模型（第 1 层），它将从基础模型的预测中学习。
meta_model = LogisticRegression(max_iter=1000)

# 创建 Stacking 分类器。
# 它会训练基础模型，然后在它们的输出上训练元模型。
# passthrough=True 表示原始特征也会传递给元模型。
stacking_clf = StackingClassifier(
    estimators=estimators,
    final_estimator=meta_model,
    passthrough=True,
    cv=5  # 用于训练基础模型的内部交叉验证
)

# --- 7. 交叉验证和预测 ---
# 设置分层 K 折交叉验证，以在每个折中保持目标分布。
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
test_predictions = np.zeros(len(X_test))

# 循环遍历每个折以进行训练和验证。
for fold, (train_index, val_index) in enumerate(skf.split(X, y)):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    # 在训练折上训练 Stacking 分类器。
    stacking_clf.fit(X_train, y_train)

    # 在验证集上进行预测以评估性能。
    y_pred_proba = stacking_clf.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, y_pred_proba)
    # 如果需要，您可以在此处添加其他指标，如准确率。

    # 累积对测试集的预测。每个折都会对最终预测做出贡献。
    test_predictions += stacking_clf.predict_proba(X_test)[:, 1] / skf.n_splits

# --- 8. 生成提交文件 ---
# 使用 0.5 的阈值将最终的平均概率转换为类别标签（0 或 1）。
final_predictions_int = (test_predictions >= 0.5).astype(int)

# 将数值标签映射回原始的字符串标签。
label_map = {1: 'Extrovert', 0: 'Introvert'}
final_predictions_labeled = pd.Series(final_predictions_int).map(label_map)

# 以所需格式创建提交的 DataFrame。
submission = pd.DataFrame({'id': test_df['id'], 'Personality': final_predictions_labeled})
submission.to_csv('submission.csv', index=False)


KeyboardInterrupt: 

In [4]:
import numpy as np
import pandas as pd
from collections import Counter

from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, accuracy_score

import warnings
warnings.filterwarnings('ignore')

# --- 1. 加载数据 ---
# 加载训练集、测试集和外部原始数据集。
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
orig_df = pd.read_csv('personality_datasert.csv')

# --- 2. 特征工程 ---
# 定义用于合并的列。
df_cols = [
    'Time_spent_Alone', 'Stage_fear', 'Social_event_attendance',
    'Going_outside', 'Drained_after_socializing',
    'Friends_circle_size', 'Post_frequency'
]

# 通过重命名列和删除重复项来准备原始数据集。
# 这会创建一个参考数据帧，用于识别训练/测试集中的匹配记录。
ref_df = (
    orig_df.rename(columns={'Personality': 'match_p'})
           .drop_duplicates(subset=df_cols)
)

# 用于将主数据帧与参考数据帧合并的函数。
# 它会添加一个 'match_p' 列（原始的人格标签）和一个标志 'match_p_null'
# 来指示是否在原始数据集中找到了匹配项。
def merge_with_match_p(df, ref_df, merge_cols):
    merged_df = df.merge(ref_df, how='left', on=merge_cols)
    merged_df['match_p_null'] = merged_df['match_p'].isna().astype(int)
    return merged_df

# 将合并函数应用于训练集和测试集。
train = merge_with_match_p(train_df, ref_df, df_cols)
test = merge_with_match_p(test_df, ref_df, df_cols)

# 删除合并过程中可能产生的任何重复行。
train = train.drop_duplicates(subset=['id'], keep='first')
test = test.drop_duplicates(subset=['id'], keep='first')

# 用 'unknown' 填充新的 'match_p' 列中的缺失值。
train['match_p'] = train['match_p'].fillna('unknown')
test['match_p'] = test['match_p'].fillna('unknown')

# --- 3. 数据预处理 ---
# 将目标变量 'Personality' 转换为二进制（1 代表外向，0 代表内向）。
train['Personality'] = train['Personality'].map({'Extrovert': 1, 'Introvert': 0})

# 定义要编码的分类列。
cat_cols = ['match_p', 'Stage_fear', 'Drained_after_socializing']

# 用于将分类字符串值映射为数值的函数。
def cat_encode(df, columns):
    for col in columns:
        if col == 'match_p':
            df[col] = df[col].map({'Extrovert': 2, 'Introvert': 1, 'unknown': 0})
        else:
            df[col] = df[col].map({'Yes': 1, 'No': 0})
    return df

# 将编码应用于训练集和测试集。
train = cat_encode(train, cat_cols)
test = cat_encode(test, cat_cols)

# 使用每列的平均值来填补数值列中的缺失值。
for col in train.drop(columns=['id', 'Personality']).columns:
    train[col] = train[col].fillna(train[col].mean())

for col in test.drop(columns=['id']).columns:
    test[col] = test[col].fillna(test[col].mean())

# --- 4. 准备模型数据 ---
# 分离特征（X）和目标（y）。
X = train.drop(['Personality', 'id'], axis=1)
y = train['Personality']
X_test = test.drop('id', axis=1)

# 计算 'scale_pos_weight' 以处理某些模型中的类别不平衡问题。
# 这会在训练期间为少数类别提供更多权重。
counter = Counter(y)
scale_pos_weight = counter[0] / counter[1]

# --- 5. 模型配置 ---
# 为基础模型定义超参数。
catboost_params = {
    'iterations': 1000, 'learning_rate': 0.001, 'depth': 8, 'l2_leaf_reg': 3,
    'border_count': 128, 'bagging_temperature': 1, 'random_strength': 1,
    'loss_function': 'Logloss', 'eval_metric': 'AUC', 'scale_pos_weight': scale_pos_weight,
    'verbose': 0, 'random_state': 42
}

lgb_params = {
    'objective': 'binary', 'n_estimators': 300, 'max_depth': 10, 'min_child_samples': 10,
    'num_leaves': 20, 'learning_rate': 0.0109, 'colsample_bytree': 0.8819,
    'subsample': 0.7015, 'scale_pos_weight': scale_pos_weight, 'metric': 'AUC',
    'random_state': 42, 'verbosity': 0
}

xgb_params = {
    'n_estimators': 642, 'learning_rate': 0.1337, 'max_depth': 8, 'subsample': 0.5003,
    'colsample_bytree': 0.5300, 'gamma': 2.3324, 'reg_lambda': 2.4718, 'reg_alpha': 0.1671,
    'min_child_weight': 8, 'objective': 'binary:logistic', 'eval_metric': 'auc',
    'use_label_encoder': False, 'scale_pos_weight': scale_pos_weight, 'random_state': 42
}

rf_params = {
    'n_estimators': 608, 'max_depth': 9, 'min_samples_split': 17, 'min_samples_leaf': 2,
    'max_features': 'log2', 'bootstrap': True, 'class_weight': 'balanced', 'random_state': 42
}

# --- 6. Stacking 集成模型 ---
# 定义基础模型（第 0 层）。
estimators = [
    ('lgb', LGBMClassifier(**lgb_params)),
    ('rf', RandomForestClassifier(**rf_params)),
    ('xgb', XGBClassifier(**xgb_params)),
    ('cat', CatBoostClassifier(**catboost_params))
]

# 定义元模型（第 1 层），它将从基础模型的预测中学习。
meta_model = LogisticRegression(max_iter=1000)

# 创建 Stacking 分类器。
# 它会训练基础模型，然后在它们的输出上训练元模型。
# passthrough=True 表示原始特征也会传递给元模型。
stacking_clf = StackingClassifier(
    estimators=estimators,
    final_estimator=meta_model,
    passthrough=True,
    cv=5  # 用于训练基础模型的内部交叉验证
)

# --- 7. 交叉验证和预测 ---
# 设置分层 K 折交叉验证，以在每个折中保持目标分布。
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
test_predictions = np.zeros(len(X_test))

# 循环遍历每个折以进行训练和验证。
for fold, (train_index, val_index) in enumerate(skf.split(X, y)):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    # 在训练折上训练 Stacking 分类器。
    stacking_clf.fit(X_train, y_train)

    # 在验证集上进行预测以评估性能。
    y_pred_proba = stacking_clf.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, y_pred_proba)
    # 如果需要，您可以在此处添加其他指标，如准确率。

    # 累积对测试集的预测。每个折都会对最终预测做出贡献。
    test_predictions += stacking_clf.predict_proba(X_test)[:, 1] / skf.n_splits

# --- 8. 生成提交文件 ---
# 使用 0.5 的阈值将最终的平均概率转换为类别标签（0 或 1）。
final_predictions_int = (test_predictions >= 0.5).astype(int)

# 将数值标签映射回原始的字符串标签。
label_map = {1: 'Extrovert', 0: 'Introvert'}
final_predictions_labeled = pd.Series(final_predictions_int).map(label_map)

# 以所需格式创建提交的 DataFrame。
submission = pd.DataFrame({'id': test_df['id'], 'Personality': final_predictions_labeled})
submission.to_csv('submission.csv', index=False)
